# Accessing BigBrain data using siibra

The multilevel human atlas supports multiple reference spaces. The BigBrain model, a microscopic resolution template,is a dataset in the Terabyte range at its full resolution of 20 micrometer, and cannot be handled in the same way than MRI-scale template like the ICBM 152 template. To work with its whole brain anatomy, a downsampled version is reasonable, while for appreciating the cortical layers a highe resolution makes sense. To extract grayvalue statistics, the full resolution can be accessed for regions of interest.

The best way to interactively explore the BigBrain in full detail is to use the [siibra explorer](https://bigbrain.humanbrainproject.eu) using your webbrowser. In this notebook we show some ways of working with BigBrain using only the Python interface of `siibra`.

We start by loading the `siibra` library.

In [1]:
import siibra
from nilearn import plotting,image

[siibra:INFO]  Version: 0.1a7
[siibra:WARNING]  This is a development release. Use at your own risk. Please file bugs and issues at https://github.com/FZJ-INM1-BDA/siibra-python.
Retrieving configuration 'siibra-0.1a7' for spaces         : 100%|██████████| 8/8 [00:00<00:00, 2970.47 files/s]
Retrieving configuration 'siibra-0.1a7' for parcellations  : 100%|██████████| 20/20 [00:00<00:00, 282.12 files/s]
Retrieving configuration 'siibra-0.1a7' for atlases        : 100%|██████████| 3/3 [00:00<00:00, 2629.66 files/s]


### Load the BigBrain whole-brain template

Next we instantiate the human atlas and retrieve the BigBrain template. `siibra` automatically selects the highest resolution that is feasible for download if we do not specify a target resolution. It creates a nibabel SpatialImage object, just as if a Nifti file had been loaded.

In [2]:
template = siibra.spaces.BIG_BRAIN.get_template().fetch()
type(template), template.shape

(nibabel.nifti1.Nifti1Image, (103, 116, 90, 1))

We could try a higher resolution, but `siibra` will refuse if this resolution is not available or leads to very large data volumes. Note: You can disable this restriction by using `force=True` parameter.

In [3]:
try: 
    siibra.spaces.BIG_BRAIN.get_template().fetch(resolution_mm=0.02)
except RuntimeError as e:
    pass

[siibra:ERROR]  Data request is too large (would result in an ~258.81 GByte download, the limit is 0.5).


 0.0200 mm   258.8071 GByte
 0.0400 mm    32.3566 GByte
 0.0800 mm     4.0446 GByte
 0.1600 mm     0.5062 GByte
 0.3200 mm     0.0633 GByte
 0.6400 mm     0.0080 GByte
 1.2800 mm     0.0010 GByte


### Accessing BigBrain native maps

BigBrain comes with a growing number of highly detailed 3D cytoarchitectonic maps that have been generated at the full microscopic resolution, sometimes composed of individual 2D maps in more than a thousand histological sections. As oppposed to the probabilistic cytoarchitectonic maps defined in the MNI spaces, these are individual subject maps and do not capture intersubject variability.

In [4]:
atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS
atlas.select_parcellation("cortical layers")
layers = atlas.get_map("bigbrain").fetch(resolution_mm=0.32)

[siibra:INFO]  Select "BigBrain Cortical Layers"


In [5]:
# shortcut for plotting rois on top of this template
plot_roi = lambda img,title : plotting.view_img(
    img, bg_img=template,
    title=title,
    resampling_interpolation='nearest',
    symmetric_cmap=False,cmap='Paired')

In [ ]:
plot_roi(layers,"Layer masks")

We can select regions from the maps to retrieve masks of particular regions. Here, we retrieve a mask of only layer III of BigBrain.

In [ ]:
atlas.select_region("3")
layermask = atlas.build_mask("bigbrain")
plot_roi(layermask,"Layer III")

In a similar fashion, we can retrieve a mask of a cortical cytoarchitectonic area:

In [ ]:
atlas.select_parcellation(siibra.parcellations['julich'])
atlas.select_region("hOc3v")
regionmask = atlas.build_mask("bigbrain",resolution_mm=0.32)
plot_roi(regionmask,"Area hOc3v")